In [1]:
from transformers import BitsAndBytesConfig, AutoTokenizer, TrainingArguments, AutoModelForCausalLM, pipeline
import pandas as pd
from datasets import Dataset 
import torch
from transformers import DataCollatorForSeq2Seq

base_model_name = "nl2ls_models/checkpoint-1960"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

device_map = {"": 0}
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(  ## If it fails at this line, restart the runtime and try again.
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=False,
    use_auth_token=False,
    low_cpu_mem_usage=True
)
#model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
#model.config.pretraining_tp = 1

def generate_answer(example):
    
    prompt = pipe.tokenizer.apply_chat_template(example["messages"][:2],
                                                tokenize=False,
                                                add_generation_prompt=True)
    terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
    
    outputs = pipe(prompt,
                max_new_tokens=512,
                eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_k=50,
                top_p=0.9,
                )
    generated_text = outputs[0]['generated_text']
    return {"ls": example['0'], "generated_text": generated_text}

def create_input_prompt(example):
    return {
        "messages": [
            {"role": "system","content": system_message},
            {"role": "user", "content": example["1"]},
        ]
    }

system_message = """
    Translate text into Link Specification:
    """

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id =  tokenizer.eos_token_id
    
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Read CSV files
test_df = pd.read_csv(f"datasets/few-shot-learning-dataset/limes-manipulated/test.txt", sep="\t")
# Convert DataFrame to Dataset
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(create_input_prompt)

# Generate answers for the dataset
results = test_dataset.map(generate_answer, batched=False)

f = open(f"results.txt", "w")
f_ls = open(f"ls.txt", "w")
for result in results:
    print("#############")
    print(result["generated_text"])
    print("########################")
    output = result["generated_text"].split("\n")[-1]
    f.write(f"{output}\n")
    f_ls.write(f"{result['ls']}\n")
f.close()
f_ls.close()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/upb/users/a/asepff/profiles/unix/cs/.local/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/upb/users/a/asepff/profiles/unix/cs/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

#############
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Translate text into Link Specification:<|eot_id|><|start_header_id|>user<|end_header_id|>

The link will be generated when the workfor of the source and the lecturingfor of the target have a weightedjaccard similarity of 50% or the teaching of the source and the coaching of the target have a mongeElkan similarity of 90% and the opposing of the source and the defending of the target have a jaccard similarity of 37%<|eot_id|><|start_header_id|>assistant<|end_header_id|>

AND(OR(OR(0.5,weightedJaccard(x.workfor,y.lecturingfor)|0.5,OR(0.9,mongeElkan(x.teaching,y.coaching)|0.9,0.37)|0.5)|0.5,jaccard(x.opposing,y.defending)|0.37)
########################
#############
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Translate text into Link Specification:<|eot_id|><|start_header_id|>user<|end_header_id|>

The link will be generated when the discussion of the source and the debating of the target have a NYS